In [5]:
import os
import pandas as pd
import numpy as np
import ase
from ase.io import read, write
import numpy, math, random
#from visualise import view
from ase import Atoms
import sys
#from dscribe.descriptors import SOAP
import itertools
import matplotlib.pyplot as plt
#import tqdm
from tqdm import tqdm
from copy import deepcopy

In [6]:
import time, os, pickle
from os.path import exists
import numpy as np
import pandas as pd
import multiprocessing as mp
import collections
import itertools
import rdkit
print(rdkit.__version__)
from rdkit import rdBase,Chem
from rdkit.Chem import AllChem,Draw, Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit import RDLogger
from rdkit.Chem.Draw import SimilarityMaps
#IPythonConsole.ipython_useSVG=True  #< set this to False if you want PNGs instead of SVGs
from rdkit.Chem.rdmolfiles import CanonicalRankAtoms
#from config import dir_smi_log
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem.rdmolfiles import CanonicalRankAtoms
import functools, operator, itertools
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcNumRings

2020.03.2


In [7]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle

from scipy.linalg import cholesky, cho_solve, solve_triangular
from scipy.optimize import minimize
from scipy.spatial.distance import squareform

In [8]:
df_smiles=pd.read_csv('smiles_clean.csv')

In [9]:
df_smiles

,idx,cation_smi,Compound,Spacer (interlayer) cation
0,187,[NH3+]CCc1ccc(F)cc1,[FC6H4(CH2)2NH3]2PbI4,4-fluorophenylethylammonium
1,529,[NH3+]CCc1ccc(F)cc1,[FC6H4(CH2)2NH3]2PbI4,4-fluorophenylethylammonium
2,466,CC(C[NH3+])CCC[NH3+],[NH3(CH2)2(CH3)(CH2)3NH3]PbBr4,3-aminopyridinium
3,351,C1=C(NC=[NH+]1)CC[NH3+],[C3H4N2(CH2)2NH3]PbBr4,Histammonium
4,824,[NH3+]C(c1ccccc1)C,(MPA)2PbI4??,beta-methylphenethylammonium
...,...,...,...,...
158,664,N[NH2+]C,[CH3NH2NH2]PbBr4,Methylhydrazinium
159,665,N[NH2+]C,[CH3NH2NH2]PbBr4,Methylhydrazinium
160,77,OCC[NH3+],[HO(CH2)3NH3]2PbI4,3-aminopropanol protonated
161,76,OCC[NH3+],[HO(CH2)2NH3]2PbI4,Ethanolammonium


In [10]:
def calculate_topo_distance(mol, atom1, atom2):
    distance_matix  =  Chem.GetDistanceMatrix(mol)
    return distance_matix[atom1, atom2]
def calculate_topo_distances(mol, atoms_in_outer, atoms_in_core):
    distance_matix  =  Chem.GetDistanceMatrix(mol)
    topo_distances=[]
    for atom_in_outer in atoms_in_outer:
        topo_distances.append(min([distance_matix[atom_in_outer, atom] for atom in atoms_in_core]))
    return topo_distances
def calculate_max_topo_distances(mol, atoms_in_outer, atoms_in_core):
    distance_matix  =  Chem.GetDistanceMatrix(mol)
    topo_distances=[]
    for atom_in_outer in atoms_in_outer:
        topo_distances.append(max([distance_matix[atom_in_outer, atom] for atom in atoms_in_core]))
    return topo_distances

In [11]:
def partition_mol(mol):
    pat = Chem.MolFromSmarts('[*]-[#6;R]')
    atoms_cleaved_at = [sorted(x) for x in mol.GetSubstructMatches(pat)]
    if len(atoms_cleaved_at)==0:
        return {'cores': [mol],
            'cores_atoms_ids': [list(range(mol.GetNumAtoms()))],
            'linkers': [],
            'linkers_atoms_ids': [],
            'sidegroups': [],
            'sidegroups_atoms_ids': [],
            'atomids_pairs_cleaved': [],
            'bondids_pairs_cleaved': []}

    bonds_cleave = [mol.GetBondBetweenAtoms(at[0], at[1]) for at in atoms_cleaved_at]
    bonds_cleave = [b for b in bonds_cleave if not b.IsInRing()]
    atoms_cleaved_at2 = [(b.GetBeginAtomIdx(), b.GetEndAtomIdx()) for b in bonds_cleave]
    bonds_cleave = [b.GetIdx() for b in bonds_cleave]


    if len(atoms_cleaved_at2)==0:
        return {'cores': [mol],
            'cores_atoms_ids': [list(range(mol.GetNumAtoms()))],
            'linkers': [],
            'linkers_atoms_ids': [],
            'sidegroups': [],
            'sidegroups_atoms_ids': [],
            'atomids_pairs_cleaved': [],
            'bondids_pairs_cleaved': []}


    frag_mol = Chem.FragmentOnBonds(mol, bonds_cleave)
    frags_ids_new = Chem.GetMolFrags(frag_mol)
    new_mols = Chem.GetMolFrags(frag_mol, asMols=True)

    resdict={}
    resdict['cores'] = []
    resdict['cores_atoms_ids'] = []
    resdict['linkers'] = []
    resdict['linkers_atoms_ids'] = []
    resdict['sidegroups'] = []
    resdict['sidegroups_atoms_ids'] = []
    resdict['atomids_pairs_cleaved'] = atoms_cleaved_at2
    resdict['bondids_pairs_cleaved'] = bonds_cleave

    count_connectors_list = [Chem.MolToSmiles(m).count('*') for m in new_mols]
    ring_infos = [m.GetRingInfo().AtomRings() for m in new_mols]

    for i,m in enumerate(new_mols):
        if count_connectors_list[i]==1: # can be sidegroup or core
            if len(ring_infos[i]) > 0: #core
                resdict['cores'].append(new_mols[i])
                resdict['cores_atoms_ids'].append(frags_ids_new[i])
            else:
                resdict['sidegroups'].append(new_mols[i])
                resdict['sidegroups_atoms_ids'].append(frags_ids_new[i])

        if count_connectors_list[i]>1: # can be linker or core
            if len(ring_infos[i]) > 0: #core
                resdict['cores'].append(new_mols[i])
                resdict['cores_atoms_ids'].append(frags_ids_new[i])
            else: #linker
                resdict['linkers'].append(new_mols[i])
                resdict['linkers_atoms_ids'].append([x for x in frags_ids_new[i] if x<mol.GetNumAtoms()])

    return resdict

In [12]:
def get_num_heteroatoms(mol_rdkit):
    ''' Count Heteroatoms (other than C,H) in structure '''
    listidx = range(mol_rdkit.GetNumAtoms())
    num_het=0
    for idx in listidx: #list(set(np.array(atom_id_lists_rings).flatten())):
        at=mol_rdkit.GetAtomWithIdx(idx).GetSymbol()
        if at != 'H' and at != 'C':
            num_het+=1
    return num_het


def get_non_ring_atoms(mol_rdkit):
    ''' Find all carbon atoms in linkers '''
    linker_atom_idx=[]
    for at in mol_rdkit.GetAtoms():
        if not at.IsInRing(): # or not at.GetIsAromatic():
            linker_atom_idx.append(at.GetIdx())
    return linker_atom_idx
def get_ring_atoms(mol_rdkit):
    ''' Find all carbon atoms in linkers '''
    linker_atom_idx=[]
    for at in mol_rdkit.GetAtoms():
        if at.IsInRing(): # or not at.GetIsAromatic():
            linker_atom_idx.append(at.GetIdx())
    return linker_atom_idx

In [13]:
def GetRingSystems(mol, includeSpiro=False):
    ri = mol.GetRingInfo()
    systems = []
    for ring in ri.AtomRings():
        ringAts = set(ring)
        nSystems = []
        for system in systems:
            nInCommon = len(ringAts.intersection(system))
            if nInCommon and (includeSpiro or nInCommon>1):
                ringAts = ringAts.union(system)
            else:
                nSystems.append(system)
        nSystems.append(ringAts)
        systems = nSystems
    return systems

In [14]:
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

In [15]:
def get_smi_features(smi):
    mol=Chem.MolFromSmiles(smi)
    if mol==None: return None
    features=[]
    feature1=get_num_heteroatoms(mol)
    features.append(feature1)
    parts=partition_mol(mol)
    Ns=[]
    #substructure = Chem.MolFromSmarts('[N+]')
    for N in mol.GetSubstructMatches(Chem.MolFromSmarts('[N+]')):
        Ns.append(N[0])
    for N in mol.GetSubstructMatches(Chem.MolFromSmarts('[n+]')):
        Ns.append(N[0])
    if len(get_ring_atoms(mol))>0:
        feature2=np.mean([calculate_topo_distances(mol, [N], get_ring_atoms(mol)) for N in Ns])
    else:
        #feature2=0
        max_dists=[]
        for N in Ns:
            max_dist=max([calculate_topo_distances( mol, [N], [i] )[0] for i in range(mol.GetNumAtoms())])
            max_dists.append(max_dist)
        feature2=np.mean(max_dists)
    features.append(feature2)
    feature3=max([CalcNumRings(x) for x in parts['cores']])
    features.append(feature3)
    
    #feature4: R-acene
    R_acene=len(parts['sidegroups_atoms_ids'])
    features.append(R_acene)
    
    #feature5: number of atoms of alkyl
    if len(parts['sidegroups_atoms_ids'])>0:
        number_alkyls=[]
        for i, group in enumerate(parts['sidegroups_atoms_ids']):
            if np.any([N in group for N in Ns]):
                alkyl_mol=parts['sidegroups'][i]
                listidx = range(alkyl_mol.GetNumAtoms())
                num_at=0
                for idx in listidx: #list(set(np.array(atom_id_lists_rings).flatten())):
                    at=alkyl_mol.GetAtomWithIdx(idx).GetSymbol()
                    if at!='*':
                        num_at+=1
                
                number_alkyls.append(num_at)
        sum_number_alkyls=sum( number_alkyls )
        features.append(sum_number_alkyls)
    else:
        features.append(mol.GetNumAtoms())
    
    
    #feature6: STEI
    STEIs=[]
    listidx = range(mol.GetNumAtoms())
    for N in Ns:
        STEI=0
        for idx in listidx:
            if idx!=N:
                STEI += 1/calculate_topo_distance(mol, N, idx)**3
        STEIs.append(STEI)
    STEI=np.mean(STEIs)
    features.append(STEI)
    
    #feature 6: alkyl-STEI
    alkyl_STEIs=[]
    for i, group in enumerate(parts['sidegroups_atoms_ids']):
        alkyl_Ns=[]
        #alkyl_mol=None
        if np.any([N in group for N in Ns]):
            alkyl_mol=parts['sidegroups'][i]

            for N_ in alkyl_mol.GetSubstructMatches(Chem.MolFromSmarts('[N+]')):
                alkyl_Ns.append(N_[0])
            for N_ in alkyl_mol.GetSubstructMatches(Chem.MolFromSmarts('[n+]')):
                alkyl_Ns.append(N_[0])
        #if alkyl_mol!=None:
            one_alkyl_STEIs=[]
            listidx = range(alkyl_mol.GetNumAtoms())
            for N_ in alkyl_Ns:
                alkyl_STEI=0
                for idx in listidx:
                    at=alkyl_mol.GetAtomWithIdx(idx).GetSymbol()
                    if at!='*' and idx!=N_:
                        alkyl_STEI += 1/calculate_topo_distance(alkyl_mol, N_, idx)**3
                one_alkyl_STEIs.append(alkyl_STEI)
            
                alkyl_STEIs.append(np.mean(one_alkyl_STEIs))
    if len(alkyl_STEIs)>0:
        mean_alkyl_STEI=np.mean(alkyl_STEIs)
        features.append(mean_alkyl_STEI)
    else:
        features.append(np.mean(STEIs))
        
    #feature7
    features.append(max([calculate_max_topo_distances(mol, [N], range(mol.GetNumAtoms()))[0] for N in Ns]))
    
    #fetaure8
    ringSys = GetRingSystems(mol)
    if len(ringSys)>0:
        features.append(len(max(ringSys, key=len)))
    else:
        features.append(0)
    
    return features

In [16]:
smiles=df_smiles.cation_smi.to_list()
data=[]
for i, row in df_smiles.iterrows():
    print(row.cation_smi)
    features=get_smi_features(row.cation_smi)
    if features!=None:
        data.append([row.idx]+ features)

 [NH3+]CCc1ccc(F)cc1
 [NH3+]CCc1ccc(F)cc1
 CC(C[NH3+])CCC[NH3+]
C1=C(NC=[NH+]1)CC[NH3+]
[NH3+]C(c1ccccc1)C
[NH3+]Cc1c[nH+]ccc1
[NH3+]Cc1c[nH+]ccc1
[NH3+]Cc1c[nH+]ccc1
[NH3+]Cc1ccc(C)cc1
[NH3+]Cc1ccc(C)cc1
[NH3+]Cc1ccc(C)cc1
[NH3+]Cc1ccc(Cl)cc1
[NH3+]CCCC(=O)O
[NH3+]CCCC(=O)O
[NH3+]CCCC1=CC=C(S1)C2=CC=CS2
[NH3+]CCCCCC[NH3+]
[NH3+]CCCCCC[NH3+]
C1=CN(C=[NH+]1)CCC[NH3+]
[NH3+]CCCn1cc[nH+]c1
[NH3+]CCSSCC[NH3+]
BrCC[NH3+]
C#CCC[NH3+]
C(C[NH3+])C#N
C(C1=CC2=CC=CC=C2C=C1)[NH3+]
C(CC[NH3+])C[NH3+]
C(CO)[NH3+]
C(CO)[NH3+]
C(CO)[NH3+]
C(COCCOCC[NH3+])[NH3+]
C(CSC(N)N)[NH3+]
[NH3+]CCSSCC[NH3+]
c(F)1c(F)c(C[NH3+])c(F)c(F)c1C[NH3+]
c(F)1c(F)c(C[NH3+])c(F)c(F)c1C[NH3+]
C[NH+](C)c1ccc(cc1)[NH+](C)C
C[NH+](C)c1ccc(cc1)[NH+](C)C
C[NH+](C)c1ccc([NH3+])cc1
C[NH+](C)c1ccc([NH3+])cc1
C[NH+](C)CC[NH+](C)C
C[NH+](C)CCC[NH3+]
C[NH+](C)CCC[NH3+]
C[NH+](C)CCC[NH3+]
C[NH+](C)CCC[NH3+]
C1(=[NH+]C(=[NH+]C(=N1)N)N)N
C1(NC=[NH+]3)=C3C=C2C(=C1)NC=[NH+]2
C1=[NH+]C(CC[NH3+])=CN1
C1=C[NH+]=C(N1)C2=[NH+]C=CN2
c1ccc(Br)c[n

In [17]:
data

[[187, 2, 3.0, 1, 2, 3, 1.2168321185617104, 1.125, 7.0, 6],
 [529, 2, 3.0, 1, 2, 3, 1.2168321185617104, 1.125, 7.0, 6],
 [466, 2, 6.0, 0, 0, 8, 1.2128101851851851, 1.2128101851851851, 6.0, 0],
 [351, 3, 1.5, 1, 1, 3, 1.7599745370370372, 1.125, 5.0, 5],
 [824, 1, 2.0, 1, 1, 3, 1.363324074074074, 1.125, 5.0, 6],
 [327, 2, 1.0, 1, 1, 2, 1.7890115740740742, 1.0, 5.0, 6],
 [329, 2, 1.0, 1, 1, 2, 1.7890115740740742, 1.0, 5.0, 6],
 [331, 2, 1.0, 1, 1, 2, 1.7890115740740742, 1.0, 5.0, 6],
 [634, 1, 2.0, 1, 2, 2, 1.2429537037037035, 1.0, 6.0, 6],
 [635, 1, 2.0, 1, 2, 2, 1.2429537037037035, 1.0, 6.0, 6],
 [636, 1, 2.0, 1, 2, 2, 1.2429537037037035, 1.0, 6.0, 6],
 [225, 2, 2.0, 1, 2, 2, 1.2429537037037035, 1.0, 6.0, 6],
 [684, 3, 5.0, 0, 0, 7, 1.193662037037037, 1.193662037037037, 5.0, 0],
 [439, 3, 5.0, 0, 0, 7, 1.193662037037037, 1.193662037037037, 5.0, 0],
 [675, 3, 4.0, 1, 1, 4, 1.2124864824163055, 1.162037037037037, 9.0, 5],
 [38, 2, 7.0, 0, 0, 8, 1.1932071185617104, 1.1932071185617104, 7.0, 

In [18]:
smi_features=pd.DataFrame(data,columns=['idx','number_heteroatoms','length_alkyl','number_rings_acene','R_acene',\
                                        'number_alkylatoms', 'STEI','alkyl_STEI','Eccentricity','LRS'])

In [19]:
smi_features

,idx,number_heteroatoms,length_alkyl,number_rings_acene,R_acene,number_alkylatoms,STEI,alkyl_STEI,Eccentricity,LRS
0,187,2,3.0,1,2,3,1.216832,1.125000,7.0,6
1,529,2,3.0,1,2,3,1.216832,1.125000,7.0,6
2,466,2,6.0,0,0,8,1.212810,1.212810,6.0,0
3,351,3,1.5,1,1,3,1.759975,1.125000,5.0,5
4,824,1,2.0,1,1,3,1.363324,1.125000,5.0,6
...,...,...,...,...,...,...,...,...,...,...
158,664,2,1.0,0,0,3,2.000000,2.000000,1.0,0
159,665,2,1.0,0,0,3,2.000000,2.000000,1.0,0
160,77,2,3.0,0,0,4,1.162037,1.162037,3.0,0
161,76,2,3.0,0,0,4,1.162037,1.162037,3.0,0


In [20]:
smi_features.to_csv('smi_features_clean.csv',index=False)